[View in Colaboratory](https://colab.research.google.com/github/andersy005/video-game-graphics-to-reality-and-back/blob/master/image_translation.ipynb)

In [1]:
!pip install Pillow==4.0.0
!pip install image
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch



  Using cached Pillow-4.0.0-cp36-cp36m-manylinux1_x86_64.whl
  Found existing installation: Pillow 5.1.0
    Uninstalling Pillow-5.1.0:
      Successfully uninstalled Pillow-5.1.0


In [2]:
torch.cuda.is_available()

True

In [3]:
#@title Default title text
!wget --header 'Host: doc-0o-58-docs.googleusercontent.com' --user-agent 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:59.0) Gecko/20100101 Firefox/59.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8' --header 'Accept-Language: en-US,en;q=0.5' --referer 'https://drive.google.com/drive/u/1/folders/1B56WZ20ODejt7MAwFr0a2HnMJ0fz4VlN' --header 'Cookie: AUTH_e1273drrbkihn7uunefac5g2nh6fppao=07610261954870412584|1523390400000|kf36r1s9jup16ems1p1f2sckdgrlvbhm; NID=123=lBPWjtG4k1sU7Dd0qsq5jpbqEiDrMBU8x0r6kdqXBCf-pCwxt9y7UdTTVaEFzLR7u5w-wq6dp0xKDTIsDb6QCnL0jNICERWUAuRHrxjQlpzo8gI8bqEpJJBDhN1T2RNR' --header 'DNT: 1' --header 'Upgrade-Insecure-Requests: 1' 'https://doc-0o-58-docs.googleusercontent.com/docs/securesc/u8k4a259k8v0rijo1dv2q4t69g74544s/il57erfe3koja1994495ashvpr49mjfs/1523390400000/07610261954870412584/07610261954870412584/1gKCTPV_pCWaIEuiQ5IWP8rQSh040ezOL?h=11006290462901738094&e=download' --output-document 'city.tar.gz'
!wget --header 'Host: doc-14-58-docs.googleusercontent.com' --user-agent 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:59.0) Gecko/20100101 Firefox/59.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8' --header 'Accept-Language: en-US,en;q=0.5' --referer 'https://drive.google.com/drive/u/1/folders/1B56WZ20ODejt7MAwFr0a2HnMJ0fz4VlN' --header 'Cookie: AUTH_e1273drrbkihn7uunefac5g2nh6fppao=07610261954870412584|1523390400000|kf36r1s9jup16ems1p1f2sckdgrlvbhm; NID=123=lBPWjtG4k1sU7Dd0qsq5jpbqEiDrMBU8x0r6kdqXBCf-pCwxt9y7UdTTVaEFzLR7u5w-wq6dp0xKDTIsDb6QCnL0jNICERWUAuRHrxjQlpzo8gI8bqEpJJBDhN1T2RNR' --header 'DNT: 1' --header 'Upgrade-Insecure-Requests: 1' 'https://doc-14-58-docs.googleusercontent.com/docs/securesc/u8k4a259k8v0rijo1dv2q4t69g74544s/3le4pio11hb399ub230qat6883ggjv5n/1523390400000/07610261954870412584/07610261954870412584/1TxNprMLCTVnTWV9QIYDve5mv_DjOsCNb?h=11006290462901738094&e=download' --output-document 'gta.tar.gz'
!tar -zxvf gta.tar.gz
!tar -zxvf city.tar.gz
!ls

--2018-04-10 23:19:01--  https://doc-0o-58-docs.googleusercontent.com/docs/securesc/u8k4a259k8v0rijo1dv2q4t69g74544s/il57erfe3koja1994495ashvpr49mjfs/1523390400000/07610261954870412584/07610261954870412584/1gKCTPV_pCWaIEuiQ5IWP8rQSh040ezOL?h=11006290462901738094&e=download
Resolving doc-0o-58-docs.googleusercontent.com (doc-0o-58-docs.googleusercontent.com)... 74.125.141.132, 2607:f8b0:400c:c06::84
Connecting to doc-0o-58-docs.googleusercontent.com (doc-0o-58-docs.googleusercontent.com)|74.125.141.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/gzip]
Saving to: ‘city.tar.gz’

city.tar.gz             [ <=>                ]   3.90G   141MB/s               

city.tar.gz             [      <=>           ]   4.63G   108MB/s    in 35s     

2018-04-10 23:19:37 (135 MB/s) - ‘city.tar.gz’ saved [4975424256]

--2018-04-10 23:19:38--  https://doc-14-58-docs.googleusercontent.com/docs/securesc/u8k4a259k8v0rijo1dv2q4t69g74544s/3le4pio11hb399ub230qat6883ggjv5n/1523390400000/07610261954870412584/07610261954870412584/1TxNprMLCTVnTWV9QIYDve5mv_DjOsCNb?h=11006290462901738094&e=download
Resolving doc-14-58-docs.googleusercontent.com (doc-14-58-docs.googleusercontent.com)... 74.125.141.132, 2607:f8b0:400c:c06::84
Connecting to doc-14-58-docs.googleusercontent.com (doc-14-58-docs.googleusercontent.com)|74.125.141.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/gzip]
Saving to: ‘gta.tar.gz’

gta.tar.gz              [       <=>          ]   3.10G   140MB/s               

gta.tar.gz              [              <=>   ]   5.31G   135MB/s    in 39s     

2018-04-10 23:20:17 (140 MB/s) - ‘gta.tar.gz’ saved [5703604814]

./gta/
./gta/images/
./gta/images/23763.png
./gta/images/24852.png
./gta/images/22810.png
./gta/images/24116.png
./gta/images/24392.png
./gta/images/24087.png
./gta/images/22576.png
./gta/images/23821.png
./gta/images/22914.png
./gta/images/23830.png
./gta/images/24539.png
./gta/images/24721.png
./gta/images/23568.png
./gta/images/22870.png
./gta/images/23220.png
./gta/images/24164.png
./gta/images/23002.png
./gta/images/22887.png
./gta/images/24416.png
./gta/images/22705.png
./gta/images/22522.png
./gta/images/22876.png
./gta/images/23738.png
./gta/images/22835.png
./gta/images/22569.png
./gta/images/24867.png
./gta/images/22995.png
./gta/images/23433.png
./gta/images/23588.png
./gta/images/23170.png
./gta/images/23475.png
./gta/images/23865.png
./gta/images/24627.png
./gta/images/24493.png
./gta/images/24831.png
./gta/images/22541.png
./gt

./gta/images/24262.png
./gta/images/24201.png
./gta/images/24535.png
./gta/images/22584.png
./gta/images/24594.png
./gta/images/24853.png
./gta/images/23026.png
./gta/images/23604.png
./gta/images/23854.png
./gta/images/24511.png
./gta/images/23726.png
./gta/images/22873.png
./gta/images/22851.png
./gta/images/24621.png
./gta/images/23734.png
./gta/images/22950.png
./gta/images/23315.png
./gta/images/24787.png
./gta/images/24197.png
./gta/images/23146.png
./gta/images/22552.png
./gta/images/23887.png
./gta/images/23650.png
./gta/images/23859.png
./gta/images/23805.png
./gta/images/24655.png
./gta/images/24952.png
./gta/images/24718.png
./gta/images/23788.png
./gta/images/23603.png
./gta/images/23579.png
./gta/images/23752.png
./gta/images/24491.png
./gta/images/23448.png
./gta/images/24250.png
./gta/images/22684.png
./gta/images/23254.png
./gta/images/24750.png
./gta/images/24033.png
./gta/images/24041.png
./gta/images/22595.png
./gta/images/24483.png
./gta/images/24507.png
./gta/image

./gta/images/23388.png
./gta/images/23382.png
./gta/images/24726.png
./gta/images/24367.png
./gta/images/24537.png
./gta/images/24011.png
./gta/images/24145.png
./gta/images/23349.png
./gta/images/23608.png
./gta/images/23936.png
./gta/images/24354.png
./gta/images/23867.png
./gta/images/24963.png
./gta/images/22694.png
./gta/images/23309.png
./gta/images/23414.png
./gta/images/24365.png
./gta/images/22655.png
./gta/images/24679.png
./gta/images/23736.png
./gta/images/22897.png
./gta/images/22762.png
./gta/images/23587.png
./gta/images/24555.png
./gta/images/24299.png
./gta/images/23115.png
./gta/images/23905.png
./gta/images/23696.png
./gta/images/23239.png
./gta/images/23041.png
./gta/images/24509.png
./gta/images/23259.png
./gta/images/23663.png
./gta/images/23278.png
./gta/images/23089.png
./gta/images/24102.png
./gta/images/22987.png
./gta/images/23619.png
./gta/images/24467.png
./gta/images/22647.png
./gta/images/24916.png
./gta/images/23403.png
./gta/images/22850.png
./gta/image

...

./city_real/strasbourg_000000_006483_leftImg8bit.png
./city_real/cologne_000122_000019_leftImg8bit.png
./city_real/strasbourg_000000_018153_leftImg8bit.png
./city_real/bremen_000238_000019_leftImg8bit.png
./city_real/stuttgart_000091_000019_leftImg8bit.png
./city_real/bochum_000000_024717_leftImg8bit.png
./city_real/bochum_000000_014658_leftImg8bit.png
./city_real/hanover_000000_055592_leftImg8bit.png
./city_real/bremen_000143_000019_leftImg8bit.png
./city_real/aachen_000029_000019_leftImg8bit.png
./city_real/strasbourg_000001_012956_leftImg8bit.png
./city_real/strasbourg_000001_026355_leftImg8bit.png
./city_real/bochum_000000_024196_leftImg8bit.png
./city_real/hanover_000000_030276_leftImg8bit.png
./city_real/aachen_000034_000019_leftImg8bit.png
./city_real/cologne_000041_000019_leftImg8bit.png
./city_real/stuttgart_000170_000019_leftImg8bit.png
./city_real/dusseldorf_000157_000019_leftImg8bit.png
./city_real/hamburg_000000_065983_leftImg8bit.png
./city_real/hamburg_000000_089696_left

./city_real/aachen_000040_000019_leftImg8bit.png
./city_real/strasbourg_000001_049776_leftImg8bit.png
./city_real/jena_000043_000019_leftImg8bit.png
./city_real/jena_000014_000019_leftImg8bit.png
./city_real/hanover_000000_001173_leftImg8bit.png
./city_real/aachen_000050_000019_leftImg8bit.png
./city_real/hamburg_000000_065843_leftImg8bit.png
./city_real/strasbourg_000000_031223_leftImg8bit.png
./city_real/hanover_000000_056142_leftImg8bit.png
./city_real/aachen_000078_000019_leftImg8bit.png
./city_real/dusseldorf_000079_000019_leftImg8bit.png
./city_real/erfurt_000106_000019_leftImg8bit.png
./city_real/dusseldorf_000006_000019_leftImg8bit.png
./city_real/strasbourg_000001_008310_leftImg8bit.png
./city_real/jena_000029_000019_leftImg8bit.png
./city_real/strasbourg_000001_041215_leftImg8bit.png
./city_real/erfurt_000094_000019_leftImg8bit.png
./city_real/strasbourg_000000_019617_leftImg8bit.png
./city_real/strasbourg_000001_032660_leftImg8bit.png
./city_real/dusseldorf_000069_000019_lef

./city_real/jena_000025_000019_leftImg8bit.png
./city_real/dusseldorf_000057_000019_leftImg8bit.png
./city_real/strasbourg_000001_007864_leftImg8bit.png
./city_real/stuttgart_000154_000019_leftImg8bit.png
./city_real/hamburg_000000_088939_leftImg8bit.png
./city_real/jena_000026_000019_leftImg8bit.png
./city_real/bremen_000060_000019_leftImg8bit.png
./city_real/hamburg_000000_047057_leftImg8bit.png
./city_real/cologne_000082_000019_leftImg8bit.png
./city_real/hamburg_000000_104428_leftImg8bit.png
./city_real/hanover_000000_047870_leftImg8bit.png
./city_real/aachen_000068_000019_leftImg8bit.png
./city_real/hanover_000000_052729_leftImg8bit.png
./city_real/erfurt_000041_000019_leftImg8bit.png
./city_real/hamburg_000000_045437_leftImg8bit.png
./city_real/bremen_000193_000019_leftImg8bit.png
./city_real/bremen_000269_000019_leftImg8bit.png
./city_real/bremen_000040_000019_leftImg8bit.png
./city_real/bremen_000067_000019_leftImg8bit.png
./city_real/hanover_000000_016558_leftImg8bit.png
./cit

./city_real/cologne_000092_000019_leftImg8bit.png
./city_real/aachen_000125_000019_leftImg8bit.png
./city_real/strasbourg_000001_059433_leftImg8bit.png
./city_real/cologne_000093_000019_leftImg8bit.png
./city_real/dusseldorf_000168_000019_leftImg8bit.png
./city_real/hamburg_000000_087822_leftImg8bit.png
./city_real/hanover_000000_034720_leftImg8bit.png
./city_real/bremen_000200_000019_leftImg8bit.png
./city_real/hamburg_000000_080438_leftImg8bit.png
./city_real/stuttgart_000139_000019_leftImg8bit.png
./city_real/strasbourg_000000_031067_leftImg8bit.png
./city_real/stuttgart_000085_000019_leftImg8bit.png
./city_real/cologne_000094_000019_leftImg8bit.png
./city_real/stuttgart_000169_000019_leftImg8bit.png
./city_real/dusseldorf_000092_000019_leftImg8bit.png
./city_real/jena_000109_000019_leftImg8bit.png
./city_real/strasbourg_000001_045135_leftImg8bit.png
./city_real/jena_000031_000019_leftImg8bit.png
./city_real/hanover_000000_052512_leftImg8bit.png
./city_real/bremen_000305_000019_left

./city_real/hamburg_000000_054555_leftImg8bit.png
./city_real/erfurt_000024_000019_leftImg8bit.png
./city_real/strasbourg_000001_008576_leftImg8bit.png
./city_real/stuttgart_000034_000019_leftImg8bit.png
./city_real/hamburg_000000_103367_leftImg8bit.png
./city_real/hanover_000000_012675_leftImg8bit.png
./city_real/hanover_000000_018213_leftImg8bit.png
./city_real/bremen_000095_000019_leftImg8bit.png
./city_real/strasbourg_000001_001449_leftImg8bit.png
./city_real/hamburg_000000_102379_leftImg8bit.png
./city_real/strasbourg_000001_052050_leftImg8bit.png
./city_real/stuttgart_000179_000019_leftImg8bit.png
./city_real/strasbourg_000000_028556_leftImg8bit.png
1i1FTiyrgv8yGF8XCvABp8WdpWHpH3T5r  city.tar.gz	gta	    models   wget-log
city_real			   datalab	gta.tar.gz  samples


In [0]:
import sys
import torch
from torchvision import datasets
from torchvision import transforms
import os
import matplotlib.pyplot as plt

In [0]:
import torch.nn as nn
import torch.nn.functional as F


def deconv(c_in, c_out, k_size, stride=2, pad=1, bn=True):
    """Custom deconvolutional layer for simplicity."""
    layers = []
    layers.append(nn.ConvTranspose2d(c_in, c_out, k_size, stride, pad, bias=False))
    if bn:
        layers.append(nn.BatchNorm2d(c_out))
    return nn.Sequential(*layers)

def conv(c_in, c_out, k_size, stride=2, pad=1, bn=True):
    """Custom convolutional layer for simplicity."""
    layers = []
    layers.append(nn.Conv2d(c_in, c_out, k_size, stride, pad, bias=False))
    if bn:
        layers.append(nn.BatchNorm2d(c_out))
    return nn.Sequential(*layers)

class G12(nn.Module):
    """Generator for transfering from mnist to svhn"""
    def __init__(self, conv_dim=64):
        super(G12, self).__init__()
        # encoding blocks
        self.conv1 = conv(3, conv_dim, 4)
        self.conv2 = conv(conv_dim, conv_dim*2, 4)
        
        # residual blocks
        self.conv3 = conv(conv_dim*2, conv_dim*2, 3, 1, 1)
        self.conv4 = conv(conv_dim*2, conv_dim*2, 3, 1, 1)
        
        # decoding blocks
        self.deconv1 = deconv(conv_dim*2, conv_dim, 4)
        self.deconv2 = deconv(conv_dim, 3, 4, bn=False)
        
    def forward(self, x):
        out = F.leaky_relu(self.conv1(x), 0.05)      # (?, 64, 16, 16)
        out = F.leaky_relu(self.conv2(out), 0.05)    # (?, 128, 8, 8)
        
        out = F.leaky_relu(self.conv3(out), 0.05)    # ( " )
        out = F.leaky_relu(self.conv4(out), 0.05)    # ( " )
        
        out = F.leaky_relu(self.deconv1(out), 0.05)  # (?, 64, 16, 16)
        out = F.tanh(self.deconv2(out))              # (?, 3, 32, 32)
        return out
    
class G21(nn.Module):
    """Generator for transfering from svhn to mnist"""
    def __init__(self, conv_dim=64):
        super(G21, self).__init__()
        # encoding blocks
        self.conv1 = conv(3, conv_dim, 4)
        self.conv2 = conv(conv_dim, conv_dim*2, 4)
        
        # residual blocks
        self.conv3 = conv(conv_dim*2, conv_dim*2, 3, 1, 1)
        self.conv4 = conv(conv_dim*2, conv_dim*2, 3, 1, 1)
        
        # decoding blocks
        self.deconv1 = deconv(conv_dim*2, conv_dim, 4)
        self.deconv2 = deconv(conv_dim, 3, 4, bn=False)
        
    def forward(self, x):
        out = F.leaky_relu(self.conv1(x), 0.05)      # (?, 64, 16, 16)
        out = F.leaky_relu(self.conv2(out), 0.05)    # (?, 128, 8, 8)
        
        out = F.leaky_relu(self.conv3(out), 0.05)    # ( " )
        out = F.leaky_relu(self.conv4(out), 0.05)    # ( " )
        
        out = F.leaky_relu(self.deconv1(out), 0.05)  # (?, 64, 16, 16)
        out = F.tanh(self.deconv2(out))              # (?, 1, 32, 32)
        return out
    
class D1(nn.Module):
    """Discriminator for mnist."""
    def __init__(self, conv_dim=64, use_labels=False):
        super(D1, self).__init__()
        self.conv1 = conv(3, conv_dim, 4, bn=False)
        self.conv2 = conv(conv_dim, conv_dim*2, 4)
        self.conv3 = conv(conv_dim*2, conv_dim*4, 4)
        n_out = 11 if use_labels else 1
        self.fc = conv(conv_dim*4, n_out, 4, 1, 0, False)
        
    def forward(self, x):
        out = F.leaky_relu(self.conv1(x), 0.05)    # (?, 64, 16, 16)
        out = F.leaky_relu(self.conv2(out), 0.05)  # (?, 128, 8, 8)
        out = F.leaky_relu(self.conv3(out), 0.05)  # (?, 256, 4, 4)
        out = self.fc(out).squeeze()
        return out

class D2(nn.Module):
    """Discriminator for svhn."""
    def __init__(self, conv_dim=64, use_labels=False):
        super(D2, self).__init__()
        self.conv1 = conv(3, conv_dim, 4, bn=False)
        self.conv2 = conv(conv_dim, conv_dim*2, 4)
        self.conv3 = conv(conv_dim*2, conv_dim*4, 4)
        n_out = 11 if use_labels else 1
        self.fc = conv(conv_dim*4, n_out, 4, 1, 0, False)
        
    def forward(self, x):
        out = F.leaky_relu(self.conv1(x), 0.05)    # (?, 64, 16, 16)
        out = F.leaky_relu(self.conv2(out), 0.05)  # (?, 128, 8, 8)
        out = F.leaky_relu(self.conv3(out), 0.05)  # (?, 256, 4, 4)
        out = self.fc(out).squeeze()
        return out

In [0]:
import torch
import torch.nn as nn
import torchvision
import os
import pickle
import scipy.io
import numpy as np

from torch.autograd import Variable
from torch import optim
from torchvision import transforms, utils
#from models.model import G12, G21
#from models.model import D1, D2


class Solver(object):
    def __init__(self, config, gta_loader, city_loader):
        self.gta_loader = gta_loader
        self.city_loader = city_loader
        self.g12 = None
        self.g21 = None
        self.d1 = None
        self.d2 = None
        self.g_optimizer = None
        self.d_optimizer = None
        self.use_reconst_loss = config.use_reconst_loss
        self.use_labels = config.use_labels
        self.num_classes = config.num_classes
        self.beta1 = config.beta1
        self.beta2 = config.beta2
        self.g_conv_dim = config.g_conv_dim
        self.d_conv_dim = config.d_conv_dim
        self.train_iters = config.train_iters
        self.batch_size = config.batch_size
        self.lr = config.lr
        self.log_step = config.log_step
        self.sample_step = config.sample_step
        self.sample_path = config.sample_path
        self.model_path = config.model_path
        self.build_model()
        
    def build_model(self):
        """Builds a generator and a discriminator."""
        self.g12 = G12(conv_dim=self.g_conv_dim)
        self.g21 = G21(conv_dim=self.g_conv_dim)
        self.d1 = D1(conv_dim=self.d_conv_dim, use_labels=self.use_labels)
        self.d2 = D2(conv_dim=self.d_conv_dim, use_labels=self.use_labels)
        
        g_params = list(self.g12.parameters()) + list(self.g21.parameters())
        d_params = list(self.d1.parameters()) + list(self.d2.parameters())
        
        self.g_optimizer = optim.Adam(g_params, self.lr, [self.beta1, self.beta2])
        self.d_optimizer = optim.Adam(d_params, self.lr, [self.beta1, self.beta2])
        
        if torch.cuda.is_available():
            self.g12.cuda()
            self.g21.cuda()
            self.d1.cuda()
            self.d2.cuda()
    
    def merge_images(self, sources, targets, k=10):
        _, _, h, w = sources.shape
        row = int(np.sqrt(self.batch_size))
        merged = np.zeros([3, row*h, row*w*2])
        for idx, (s, t) in enumerate(zip(sources, targets)):
            i = idx // row
            j = idx % row
            merged[:, i*h:(i+1)*h, (j*2)*h:(j*2+1)*h] = s
            merged[:, i*h:(i+1)*h, (j*2+1)*h:(j*2+2)*h] = t
        return merged.transpose(1, 2, 0)
    
    def to_var(self, x):
        """Converts numpy to variable."""
        if torch.cuda.is_available():
            x = x.cuda()
        return Variable(x)
    
    def to_data(self, x):
        """Converts variable to numpy."""
        if torch.cuda.is_available():
            x = x.cpu()
        return x.data.numpy()
    
    def reset_grad(self):
        """Zeros the gradient buffers."""
        self.g_optimizer.zero_grad()
        self.d_optimizer.zero_grad()

    def train(self):
        gta_iter = iter(self.gta_loader)
        city_iter = iter(self.city_loader)
        iter_per_epoch = min(len(gta_iter), len(city_iter))
        
        # fixed city and gta for sampling
        fixed_gta = self.to_var(gta_iter.next())
        fixed_city = self.to_var(city_iter.next())
        
        # loss if use_labels = True
        criterion = nn.CrossEntropyLoss()
        
        for step in range(self.train_iters+1):
            # reset data_iter for each epoch
            if (step+1) % iter_per_epoch == 0:
                city_iter = iter(self.city_loader)
                gta_iter = iter(self.gta_loader)
            
            # load gta and city dataset
            gta = gta_iter.next() 
            gta = self.to_var(gta)
            city = city_iter.next() 
            city = self.to_var(city)

            if self.use_labels:
                city_fake_labels = self.to_var(
                    torch.Tensor([self.num_classes]*gta.size(0)).long())
                gta_fake_labels = self.to_var(
                    torch.Tensor([self.num_classes]*city.size(0)).long())
            
            #============ train D ============#
            
            # train with real images
            self.reset_grad()
            out = self.d1(city)
            if self.use_labels:
                d1_loss = criterion(out, m_labels)
            else:
                d1_loss = torch.mean((out-1)**2)
            
            out = self.d2(gta)
            if self.use_labels:
                d2_loss = criterion(out, s_labels)
            else:
                d2_loss = torch.mean((out-1)**2)
            
            d_city_loss = d1_loss
            d_gta_loss = d2_loss
            d_real_loss = d1_loss + d2_loss
            d_real_loss.backward()
            self.d_optimizer.step()
            
            # train with fake images
            self.reset_grad()
            fake_gta = self.g12(city)
            out = self.d2(fake_gta)
            if self.use_labels:
                d2_loss = criterion(out, gta_fake_labels)
            else:
                d2_loss = torch.mean(out**2)
            
            fake_city = self.g21(gta)
            out = self.d1(fake_city)
            if self.use_labels:
                d1_loss = criterion(out, city_fake_labels)
            else:
                d1_loss = torch.mean(out**2)
            
            d_fake_loss = d1_loss + d2_loss
            d_fake_loss.backward()
            self.d_optimizer.step()
            
            #============ train G ============#
            
            # train city-gta-city cycle
            self.reset_grad()
            fake_gta = self.g12(city)
            out = self.d2(fake_gta)
            reconst_city = self.g21(fake_gta)
            if self.use_labels:
                g_loss = criterion(out, m_labels) 
            else:
                g_loss = torch.mean((out-1)**2) 

            if self.use_reconst_loss:
                g_loss += torch.mean((city - reconst_city)**2)

            g_loss.backward()
            self.g_optimizer.step()

            # train gta-city-gta cycle
            self.reset_grad()
            fake_city = self.g21(gta)
            out = self.d1(fake_city)
            reconst_gta = self.g12(fake_city)
            if self.use_labels:
                g_loss = criterion(out, s_labels) 
            else:
                g_loss = torch.mean((out-1)**2) 

            if self.use_reconst_loss:
                g_loss += torch.mean((gta - reconst_gta)**2)

            g_loss.backward()
            self.g_optimizer.step()
            
            # print the log info
            if (step+1) % self.log_step == 0:
                print('Step [%d/%d], d_real_loss: %.4f, d_city_loss: %.4f, d_gta_loss: %.4f, '
                      'd_fake_loss: %.4f, g_loss: %.4f' 
                      %(step+1, self.train_iters, d_real_loss.data[0], d_city_loss.data[0], 
                        d_gta_loss.data[0], d_fake_loss.data[0], g_loss.data[0]))

            # save the sampled images
            if (step+1) % self.sample_step == 0:
                fake_gta = self.g12(fixed_city)
                fake_city = self.g21(fixed_gta)
                
                city, fake_city = self.to_data(fixed_city), self.to_data(fake_city)
                gta , fake_gta = self.to_data(fixed_gta), self.to_data(fake_gta)
                
                merged = self.merge_images(city, fake_gta)
                path = os.path.join(self.sample_path, 'sample-%d-m-s.png' %(step+1))
                path2 = os.path.join(self.sample_path, 'sample2-%d-m-s.png' %(step+1))
                path3 = os.path.join(self.sample_path, 'sample3-%d-m-s.png' %(step+1))
                path4 = os.path.join(self.sample_path, 'sample4-%d-m-s.png' %(step+1))
                path5 = os.path.join(self.sample_path, 'sample5-%d-m-s.png' %(step+1))
               
                plt.imshow(city.transpose(0, 2, 3, 1)[0])
                plt.imshow(fake_city.transpose(0, 2, 3, 1)[0])
                plt.imshow(gta.transpose(0, 2, 3, 1)[0])
                plt.imshow(fake_gta.transpose(0, 2, 3, 1)[0])
                #scipy.misc.imsave(path, merged)
                #scipy.misc.imsave(path2, city[0])
                #scipy.misc.imsave(path3, fake_gta[0])
                #scipy.misc.imsave(path4, fake_city[0])
                #scipy.misc.imsave(path5, gta[0])
                
                print ('saved %s' %path)
                
                merged = self.merge_images(gta, fake_city)
                path = os.path.join(self.sample_path, 'sample-%d-s-m.png' %(step+1))
                scipy.misc.imsave(path, merged)
                print ('saved %s' %path)
            
            if (step+1) % 20 == 0:
                # save the model parameters for each epoch
                g12_path = os.path.join(self.model_path, 'g12-%d.pkl' %(step+1))
                g21_path = os.path.join(self.model_path, 'g21-%d.pkl' %(step+1))
                d1_path = os.path.join(self.model_path, 'd1-%d.pkl' %(step+1))
                d2_path = os.path.join(self.model_path, 'd2-%d.pkl' %(step+1))
                torch.save(self.g12.state_dict(), g12_path)
                torch.save(self.g21.state_dict(), g21_path)
                torch.save(self.d1.state_dict(), d1_path)
                torch.save(self.d2.state_dict(), d2_path)

In [0]:
# Script to load, process/augment data from GTA5 and CityScape datasets.abs
# License: MIT
# Author: Anderson Banihirwe

import torch
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from skimage import io
import pathlib
import os


class Config:
    """Configuration class that
    contains needed configuration settings."""
    def __init__(self, gta='./gta/images/',
                 city='./city_real/',
                 image_size=32, g_conv_dim=64, d_conv_dim=64,
                 use_reconst_loss=True, use_labels=False, num_classes=None,
                 train_iters=100, batch_size=64, num_workers=4, lr=0.0002,
                 beta1=0.5, beta2=0.999, mode='train', model_path='models',
                 sample_path='samples', log_step=10, sample_step=10):
        self.gta_path = gta
        self.city_path = city
        self.image_size = image_size
        self.g_conv_dim = g_conv_dim
        self.d_conv_dim = d_conv_dim
        self.use_reconst_loss = use_reconst_loss
        self.train_iters = train_iters
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.beta1 = beta1
        self.beta2 = beta2
        self.mode = mode
        self.model_path = model_path
        self.sample_path = sample_path
        self.log_step = log_step
        self.sample_step = sample_step
        self.use_labels = use_labels
        self.num_classes = num_classes
        self.lr = lr


class CustomDataset(Dataset):
    """Create a custom dataset object."""
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.files = os.listdir(root_dir)

    def __len__(self):
        return len(os.listdir(self.root_dir))

    def __getitem__(self, idx):
        img_name = self.root_dir/self.files[idx]
        image = io.imread(img_name)
        if self.transform:
            image = self.transform(image)
        return image


def get_loader(config, transfms=None):
    """Builds and returns Dataloader for GTA5 and CityScape dataset."""
    gta_path = pathlib.Path(config.gta_path)
    city_path = pathlib.Path(config.city_path)

    gta_dataset = CustomDataset(gta_path, transfms)
    city_dataset = CustomDataset(city_path, transfms)
    
    dataloader_gta = DataLoader(gta_dataset, batch_size=config.batch_size, shuffle=True, num_workers=config.num_workers)
    dataloader_city = DataLoader(city_dataset, batch_size=config.batch_size, shuffle=True, num_workers=config.num_workers)

    return dataloader_gta, dataloader_city

In [0]:
config = Config()

In [0]:
transfms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [0]:
gta_loader, city_loader = get_loader(config, transfms=transfms)
len(gta_loader)
def show_batch(sample_batched):
    images_batch = sample_batched
    batch_size = len(images_batch)
    im_size = images_batch.size(2)
    grid = utils.make_grid(images_batch)
    plt.imshow(grid.numpy().transpose((1, 2, 0)))
    

In [0]:
solver = Solver(config, gta_loader, city_loader)

In [0]:
from torch.backends import cudnn
cudnn.benchmark = True

In [0]:
# create directories if not exist
if not os.path.exists(config.model_path):
    os.makedirs(config.model_path)
if not os.path.exists(config.sample_path):
    os.makedirs(config.sample_path)
    

In [0]:
solver.train()

Step [10/100], d_real_loss: 0.2862, d_city_loss: 0.1343, d_gta_loss: 0.1519, d_fake_loss: 0.3942, g_loss: 2.0404
....... (64, 3, 64, 64)
(64, 64, 64, 3)
[[[-1.6384109  -1.4404761  -1.2989979 ]
  [-1.5870366  -1.3879551  -1.2467101 ]
  [-1.0904187  -0.810224   -0.7238344 ]
  ...
  [-1.6212862  -1.4229691  -1.2467101 ]
  [-1.4671633  -1.30042    -1.1595641 ]
  [-1.4500387  -1.352941   -1.2118517 ]]

 [[-1.6212862  -1.4404761  -1.2989979 ]
  [-1.6041614  -1.405462   -1.2641394 ]
  [-1.1246684  -0.84523803 -0.7586928 ]
  ...
  [-1.6384109  -1.4404761  -1.2815686 ]
  [-1.6041614  -1.3879551  -1.2467101 ]
  [-1.5870366  -1.3879551  -1.2467101 ]]

 [[-1.6212862  -1.4229691  -1.2815686 ]
  [-1.6041614  -1.405462   -1.2467101 ]
  [-1.1246684  -0.84523803 -0.7586928 ]
  ...
  [-1.6212862  -1.4229691  -1.2641394 ]
  [-1.5699118  -1.352941   -1.2118517 ]
  [-1.3986644  -1.1428571  -1.0375599 ]]

 ...

 [[-0.9362959  -0.635154   -0.5844008 ]
  [-0.9362959  -0.635154   -0.5844008 ]
  [-0.9362959  -0